<h1> GENRE: RAP </h1>

<h3>(1) Access the data from the dataset </h3>


In [82]:
%%capture
%pip install datasets

from datasets import load_dataset

ds = load_dataset("sebastiandizon/genius-song-lyrics")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
from heapq import nlargest
from datasets import Dataset

# Assuming the dataset has a 'category' column
unique_categories = ds['train'].unique('tag')  # Get all unique categories

# Group into subdatasets
subdatasets = {category: ds['train'].filter(lambda x: x['tag'] == category and x['language'] == 'en') for category in unique_categories}

type_of_music = 'rap'
# Group into music decades
#subdataset_70_without_sort = ds['train'].filter(lambda song: 1960 <= song["year"] <= 1970 and song['tag'] == type_of_music)

subdataset_70 = ds['train'].filter(lambda song: 1970 <= song["year"] <= 1979 and song['tag'] == type_of_music).sort('views', reverse=True)
print("70s Done")
subdataset_80 = ds['train'].filter(lambda song: 1980 <= song["year"] <= 1989 and song['tag'] == type_of_music).sort('views', reverse=True).select(range(1000))
print("80s Done")
subdataset_90 = ds['train'].filter(lambda song: 1990 <= song["year"] <= 1999 and song['tag'] == type_of_music).sort('views', reverse=True).select(range(1000))
print("90s Done")
subdataset_00 = ds['train'].filter(lambda song: 2000 <= song["year"] <= 2009 and song['tag'] == type_of_music).sort('views', reverse=True).select(range(1000))
print("00s Done")
#subdataset_10 = ds['train'].filter(lambda song: 2010 <= song["year"] <= 2020 and song['tag'] == type_of_music).sort('views', reverse=True).select(range(1000))
filtered_ds = ds['train'].filter(
    lambda song: 2010 <= song["year"] <= 2019 and song['tag'] == type_of_music,
    num_proc=4
)

# 2) Umwandeln in eine Python-Liste
all_filtered = list(filtered_ds)

# 3) Nur die Top-1000 nach 'views' herausholen
top_1000 = nlargest(1000, all_filtered, key=lambda x: x["views"])

# 4) (Optional) Zurück in ein Dataset
subdataset_10 = Dataset.from_list(top_1000)
print("10s Done")
#subdataset_30 = ds['train'].filter(lambda song: 2020 <= song["year"] <= 2030 and song['tag'] == type_of_music).sort('views', reverse=True).select(range(1000))
filtered_ds = ds['train'].filter(
    lambda song: 2020 <= song["year"] <= 2029 and song['tag'] == type_of_music,
    num_proc=4
)

# 2) Umwandeln in eine Python-Liste
all_filtered = list(filtered_ds)

# 3) Nur die Top-1000 nach 'views' herausholen
top_1000 = nlargest(1000, all_filtered, key=lambda x: x["views"])

# 4) (Optional) Zurück in ein Dataset
subdataset_20 = Dataset.from_list(top_1000)
print("20s Done")

# Create subdataset
#eng_rap = subdatasets['rap']

Filter:   0%|          | 0/5134856 [00:00<?, ? examples/s]

70s Done


Filter:   0%|          | 0/5134856 [00:00<?, ? examples/s]

80s Done


Filter:   0%|          | 0/5134856 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [83]:
import heapq
from datasets import Dataset
type_of_music= 'rap'
TOP_K = 1000
decade_heaps = {70: [], 80: [], 90: [], 00: [], 10:[], 20:[]}  # z.B. 70=70er, 0=00er etc.

def get_decade(year):
    if 1970 <= year <= 1979:
        return 70
    elif 1980 <= year <= 1989:
        return 80
    elif 1990 <= year <= 1999:
        return 90
    elif 2000 <= year <= 2009:
        return 00
    elif 2010 <= year <= 2019:
        return 10
    elif 2020 <= year <= 2029:
        return 20
    return None

i = 0  # Zähler für tie-break
for song in ds["train"]:
    if song["tag"] != type_of_music:
        continue
    elif song["language"] != "en":
        continue
    dec = get_decade(song["year"])
    if dec is None:
        continue

    views = song["views"]
    heap = decade_heaps[dec]

    # Das Tupel enthält jetzt (views, i, song).
    # Der zweite Wert i ist rein für Tie-Break (und garantiert sortierbar).
    if len(heap) < TOP_K:
        heapq.heappush(heap, (views, i, song))
    else:
        # Vergleiche nur, ob aktuelle views (oder Tie-Break i) größer sind 
        # als das Minimum im Heap (heap[0]).
        if (views, i) > (heap[0][0], heap[0][1]):
            heapq.heapreplace(heap, (views, i, song))
    i += 1

# Am Ende hat jede Dekade max. 1000 Elemente.
decade_toplists = {}
for dec, heap in decade_heaps.items():
    # Sortieren nach (views, i) absteigend.
    # Falls dir der Index egal ist, kannst du einfach nur nach views sortieren.
    sorted_heap = sorted(heap, key=lambda x: (x[0], x[1]), reverse=True)
    # x[0] = views, x[1] = tie-break index, x[2] = song
    songs_only = [item[2] for item in sorted_heap]
    decade_toplists[dec] = songs_only

# Optional: zu HF Dataset konvertieren
subdataset_70 = Dataset.from_list(decade_toplists[70])
subdataset_80 = Dataset.from_list(decade_toplists[80])
subdataset_90 = Dataset.from_list(decade_toplists[90])
subdataset_00 = Dataset.from_list(decade_toplists[00])
subdataset_10 = Dataset.from_list(decade_toplists[10])
subdataset_20 = Dataset.from_list(decade_toplists[20])

# Save to hard drive
subdataset_70.save_to_disk("subdataset_70_rap")
subdataset_80.save_to_disk("subdataset_80_rap")
subdataset_90.save_to_disk("subdataset_90_rap")
subdataset_00.save_to_disk("subdataset_00_rap")
subdataset_10.save_to_disk("subdataset_10_rap")
subdataset_20.save_to_disk("subdataset_20_rap")




Saving the dataset (0/1 shards):   0%|          | 0/217 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

<h2> LDA </h2>

<h2> Remove the bad words</h2>

In [12]:
subdataset_00[1]

{'title': 'Stan',
 'tag': 'rap',
 'artist': 'Eminem',
 'year': 2000,
 'views': 3298983,
 'features': '{Dido}',
 'lyrics': '[Produced by DJ Mark the 45 King]\n\n[Chorus: Dido]\nMy tea\'s gone cold, I\'m wondering why I\nGot out of bed at all\nThe morning rain clouds up my window\nAnd I can\'t see at all\nAnd even if I could it\'d all be gray\nBut your picture on my wall\nIt reminds me that it\'s not so bad, it\'s not so bad\nMy tea\'s gone cold, I\'m wondering why I\nGot out of bed at all\nThe morning rain clouds up my window\nAnd I can\'t see at all\nAnd even if I could it\'d all be gray\nBut your picture on my wall\nIt reminds me that it\'s not so bad, it\'s not so bad\n\n[Verse 1: Eminem]\nDear Slim, I wrote you, but you still ain\'t callin\'\nI left my cell, my pager and my home phone at the bottom\nI sent two letters back in autumn, you must not\'ve got \'em\nThere probably was a problem at the post office or somethin\'\nSometimes I scribble addresses too sloppy when I jot \'em\nBu

In [2]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("tushifire/ldnoobw")
path = kagglehub.dataset_download("sahib12/badwords")

#file = kagglehub.load_dataset("tushifire/ldnoobw",path)

print("Path to dataset files:", path)

swear_words_file = os.path.join(path, 'Terms-to-Block.csv')  # Replace 'en.txt' with the correct filename

with open(swear_words_file, 'r') as f:
    swear_words = set(f.read().splitlines())

print(f"!!!!Loaded {len(swear_words)} swear words.")

#print(swear_words)
print("Done")


Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
Done


In [3]:
import gensim
import re
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from gensim.parsing.preprocessing import remove_stopwords

# Sicherstellen, dass NLTK-Daten heruntergeladen sind
nltk.download('stopwords')
nltk.download('punkt')

used_swear_words = []
# clean the text
import re


# Stopwords definieren
stop_words = set(stopwords.words('english')+["like", "oh", "na", "la", "yo", "get", "yeah", "got","verse", "chorus", "hook", "bridge", "outro", "intro"])
print("Stopwords:", stop_words)

def clean_text(document_list):
    
    cleaned_document_list = []
    
    for single_doc in document_list:
        # Delete HTML-Tags
        text = re.sub(r"<.*?>", "", single_doc)
        # Delete Sonderzeichen und Zahlen
        text = re.sub(r"[^a-zA-Z\s]", "", text)
        # Delete uncessary whitespaces
        text = re.sub(r"\s+", " ", text).strip()
        
        # lower case the text
        text = text.lower()
        
        cleaned_document_list.append(text)
    
    return cleaned_document_list


def lemmatize_docs(subdataset):
    import spacy

    nlp = spacy.load("en_core_web_md")
    
    lemmatized_document_list = []
    lemmatized = []
    
    for single_doc in subdataset:
        nlp_doc = nlp(single_doc)

        for token in nlp_doc:
        # Nur Tokens behalten, die:
        # - kein Stopwort sind
        # - kein Satzzeichen sind
        # - eine gewisse Länge haben (z. B. > 1 Zeichen)
            if not token.is_stop and not token.is_punct and len(token.text.strip()) > 1:
                lemmatized.append(token.lemma_.lower())
        
        
        lemmatized_document_list.append(lemmatized)
    
    return lemmatized_document_list
    

def get_swear_words():

    path = kagglehub.dataset_download("tushifire/ldnoobw")
    path = kagglehub.dataset_download("sahib12/badwords")

    #file = kagglehub.load_dataset("tushifire/ldnoobw",path)

    print("Path to dataset files:", path)

    swear_words_file = os.path.join(path, 'Terms-to-Block.csv')  # Replace 'en.txt' with the correct filename

    with open(swear_words_file, 'r') as f:
        swear_words = set(f.read().splitlines())

    print(f"!!!!Loaded {len(swear_words)} swear words.")

    return swear_words

def get_manual_stop_words():
    
    path = '/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Manually_Banned_Words.txt'
    path_manually_stop_words = '/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/stopwords_en.txt'

    with open(path, 'r', encoding='utf-8') as f:
        text_banned = f.read()
    
    with open(path_manually_stop_words, 'r', encoding='utf-8') as f:
        text_stopwords = f.read()
        
    banned_words = set(text_banned.split())
    stop_words = set(text_stopwords.split())
    
    return banned_words.union(stop_words)

# Preprocessing: Tokenisieren, Stopwords entfernen, Kleinbuchstaben
def preprocess_stop_words(document_list):
    # Stopwords definieren
    
    stop_words = set(stopwords.words('english'))  # Standard-Stopwords
    #additional_words = {"like", "oh", "na", "la", "yo", "you", "get", "yeah" 
    #                       "the", "and", "of", "on", "[Intro]", "[Storyteller]"}
    
    # import the manual files of stop words
    additional_words = get_manual_stop_words()
    swear_words = get_swear_words()
      
    word_set = stop_words.union(additional_words)
    word_set = word_set.union(swear_words)
        
    replacement = ""
    list_of_tokens = []
    for single_doc in document_list:

        words = single_doc.split()
        
        # replace word if not in the list
        replaced_words = []
        for word in words:
            if word.lower() in word_set:
                replaced_words.append(replacement)  
                
                if word.lower() in swear_words:
                    used_swear_words.append(word) # save word to the used swear words list
            else:
                replaced_words.append(word)  
                
        list_of_tokens.append(str(" ".join(replaced_words)))
              
    
    return list_of_tokens

def remove_repeated_lines(doc):

    unique_lines = []
    seen = set()
    for line in doc:
        if line not in seen:
            unique_lines.append(line)
            seen.add(line)
    # Am Ende kannst du wieder eine Liste zurückgeben:
    return unique_lines

def tokenize_all_docs(document_list):
    list_of_tokens = []
    
    for single_doc in document_list:
        
        if type(single_doc) == str:
            tokens = word_tokenize(single_doc)
            list_of_tokens.append(tokens)
        else:
            continue
    
    return list_of_tokens

def get_used_swear_words():
    return used_swear_words




def preprocess(doc):
    doc = re.sub(r"[^a-zA-Z0-9\s]", " ", doc) # delete unncessary things
    #doc = remove_stopwords(doc.lower())
    doc = remove_repeated_lines(doc)
    
    tokens = word_tokenize(doc.lower())  # Kleinbuchstaben und Tokenisierung
    
    tokens = [word for word in tokens if word not in stop_words]  # Stopwords entfernen
    
    return tokens

def remove_repeated_lines(doc):
    lines = doc.split("\n")
    return "\n".join(sorted(set(lines), key=lines.index))

def sort_out_bad_words(doc,word_list, replacement=" "):

    # Erstelle ein Set für einen schnelleren Lookup
    word_set = set(word_list)
    
    # Splitte den Text in Wörter
    words = doc.split()
    
    # Ersetze jedes Wort, falls es in der Liste vorkommt
    replaced_words = [replacement if word.lower() in word_set else word for word in words]
    
    # Füge den Text wieder zusammen
    return " ".join(replaced_words)
 

def process_subdatasets(subdataset):
    # Get rid of the bad words and replace it with "swear word"
    #without_swear = [sort_out_bad_words(doc, swear_words ) for doc in subdataset]

    # Preprocessing für alle Dokumente anwenden
    #processed_docs = [preprocess(doc) for doc in without_swear]

    cleaned_dataset_no_strange_characters = clean_text(subdataset)

    cleaned_dataset_no_swears = preprocess_stop_words(cleaned_dataset_no_strange_characters)

    lemmatized_dataset = lemmatize_docs(cleaned_dataset_no_swears)
    
    #tokenized_dataset = tokenize_all_docs(lemmatized_dataset)



    
    #if stop_words in processed_docs:
    #    print("FEHLER: Stopwords nicht entfernt!")
        
    # Erstellen eines Wörterbuchs
    #dictionary = Dictionary(processed_docs)
    dictionary = Dictionary(lemmatized_dataset)
    
    # Erstellen eines Bag-of-Words-Korpus
    corpus = [dictionary.doc2bow(doc) for doc in lemmatized_dataset]

    # LDA-Modell trainieren
    lda_model = gensim.models.LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=5,  # Anzahl der Themen
        random_state=42,
        passes=40,  # Anzahl der Durchgänge
        alpha='symmetric',
        eta='auto'
        #alpha='auto'  # Dirichlet-Hyperparameter Entscheidung zwischen Auto und symetric
    )
    
    id2word = dictionary
    dict_topic = {}
    # Themen anzeigen
    #print("Themen und ihre Schlüsselwörter:")
    for idx, topic in lda_model.print_topics(num_words=5):

        dict_topic[idx] = topic 

    return dict_topic


Stopwords: {'will', 'as', 'those', 'wasn', 'aren', 'no', 'your', 'outro', 'couldn', "haven't", 'oh', 'is', 'ma', 'themselves', 'all', 'weren', "you'd", "don't", 'needn', 'if', 'yours', 'under', 'any', 'then', 'of', 'further', 'yourselves', 'and', 've', 'do', 'had', 'you', 'ain', "it's", 'this', 'about', 'how', "mustn't", 'won', 'doesn', "wasn't", 'her', 'not', "aren't", 'bridge', 'have', 'so', 'before', 'am', 'ourselves', 'more', 't', 'shouldn', 'an', 'd', 'than', 'she', 'just', 'there', "weren't", 'on', 'll', 'why', 'wouldn', 'by', 'whom', 'i', "shan't", 'that', "hasn't", 'too', 'myself', 'were', 'once', "shouldn't", 'but', 'the', "isn't", "didn't", "needn't", 'o', 'now', 'has', 'very', 'they', 'having', 'was', 'me', 'itself', 'hasn', 'for', 'below', 'chorus', 'above', 'himself', 'didn', "she's", 'be', 'into', "should've", 'theirs', 'from', 's', 'both', 'mustn', 'a', 'my', 'yourself', 'between', 'to', 'some', 'who', 'y', 'own', 'because', 'same', 'doing', "won't", 'get', 'it', 'our', 

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maurice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maurice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
amount = 1000
from datasets import load_from_disk
path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Subdatasets/rap/"


rap_70_lyrics = process_subdatasets(load_from_disk(path + "subdataset_70_rap")['lyrics'][:amount])
print("70s Done")
rap_80_lyrics = process_subdatasets(load_from_disk(path + "subdataset_80_rap")['lyrics'][:amount])
print("80s Done")
rap_90_lyrics = process_subdatasets(load_from_disk(path + "subdataset_90_rap")['lyrics'][:amount])
print("90s Done")
rap_00_lyrics = process_subdatasets(load_from_disk(path + "subdataset_00_rap")['lyrics'][:amount])
print("00s Done")
rap_10_lyrics = process_subdatasets(load_from_disk(path + "subdataset_10_rap")['lyrics'][:amount])
print("10s Done")
rap_20_lyrics = process_subdatasets(load_from_disk(path + "subdataset_20_rap")['lyrics'][:amount])
print("20s Done")
#rap_30_lyrics = process_subdatasets(subdataset_30['lyrics'][:amount])

#list_of_lyrics = [rap_70_lyrics,rap_80_lyrics]
list_of_lyrics = {
    "70s": rap_70_lyrics,
    "80s": rap_80_lyrics,
    "90s": rap_90_lyrics,
    "00s": rap_00_lyrics,
    "10s": rap_10_lyrics,
    "20s": rap_20_lyrics
}

import csv

data_for_csv = []

for decade_name, decade in list_of_lyrics.items():
    i = 0
    for topics in decade:
        i = i + 1
        data_for_csv.append({
            "Method": "LDA",
            "Genre": "Rap",
            "Decade": decade_name,
            "Topic": str(decade[topics])
        })

path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/"

# Datei schreiben
output_file = "Analysis_Folder/topics_rap_LDA_1000_11_1.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
    writer.writeheader()  # Schreibe die Kopfzeile
    writer.writerows(data_for_csv)  # Schreibe die Daten

print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")


Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
70s Done
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
80s Done
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
90s Done
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
10s Done
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
20s Done


FileNotFoundError: [Errno 2] No such file or directory: 'Analysis_Folder/topics_rap_LDA_1000_11_1.csv'

In [ ]:
path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/"
# Datei schreiben
output_file = path + "Analysis_Folder/topics_rap_LDA_1000_11_1.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
    writer.writeheader()  # Schreibe die Kopfzeile
    writer.writerows(data_for_csv)  # Schreibe die Daten

print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")

CSV-Datei '/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Analysis_Folder/topics_rap_LDA_1000_11_1.csv' wurde erfolgreich erstellt!


BiGrams


In [1]:
# Input Token

from gensim.models.phrases import Phrases, Phraser
from gensim.corpora import Dictionary
from gensim.models import LdaModel


def calculate_with_bigrams(subdataset):
    # Phrases braucht eine Liste von tokenisierten Sätzen
    # mindestens in 2 Dokuemnten min_count=2
    # threshold= strikt, wie oft ein bigram vorkommen muss (höher)
    
    cleaned_dataset_no_strange_characters = clean_text(subdataset)

    cleaned_dataset_no_swears = preprocess_stop_words(cleaned_dataset_no_strange_characters)

    lemmatized_dataset = lemmatize_docs(cleaned_dataset_no_swears)
    
    #tokenized_dataset = tokenize_all_docs(lemmatized_dataset)
    tokenized_dataset=lemmatized_dataset
    
    bigram = Phrases(tokenized_dataset, min_count=1, threshold=1)


    bigram_phraser = Phraser(bigram)

    tokenized_corpus_bigrams = [bigram_phraser[sent] for sent in tokenized_dataset]
    #for sent in tokenized_corpus_bigrams[:5]:
    #    print("Nach Bigram-Phraser:", sent)
    #    print("-----")


    # 1) Dictionary
    dictionary = Dictionary(tokenized_corpus_bigrams)

    # 2) Bag-of-Words
    corpus_bow = [dictionary.doc2bow(doc) for doc in tokenized_corpus_bigrams]

    # 3) LDA trainieren
    lda_model = LdaModel(corpus=corpus_bow,
                        id2word=dictionary,
                        num_topics=2,  # Beispiel, wähle was passt
                        passes=10,     # Mehrfache Durchläufe
                        random_state=42)

    # Ergebnis inspizieren
    #for idx, topic in lda_model.show_topics(formatted=False, num_words=5):
    #    print(f"Topic {idx}: {[word for word, _ in topic]}")
        
    dict_topic = {}
    for idx, topic in lda_model.print_topics(num_words=10):

        dict_topic[idx] = topic 

    return dict_topic
    

In [ ]:
amount = 1000
from datasets import load_from_disk

path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Subdatasets/rap/"


rap_70_lyrics = calculate_with_bigrams(load_from_disk(path + 'subdataset_70_rap')['lyrics'][:amount])
print("Done")
rap_80_lyrics = calculate_with_bigrams(load_from_disk(path + "subdataset_80_rap")['lyrics'][:amount])
print("Done")
rap_90_lyrics = calculate_with_bigrams(load_from_disk(path + "subdataset_90_rap")['lyrics'][:amount])
print("Done")
rap_00_lyrics = calculate_with_bigrams(load_from_disk(path + "subdataset_00_rap")['lyrics'][:amount])
print("Done")
rap_10_lyrics = calculate_with_bigrams(load_from_disk(path + "subdataset_10_rap")['lyrics'][:amount])
print("Done")
rap_20_lyrics = calculate_with_bigrams(load_from_disk(path + "subdataset_20_rap")['lyrics'][:amount])
print("Completly Done")

list_of_lyrics = {
    "70s": rap_70_lyrics,
    "80s": rap_80_lyrics,
    "90s": rap_90_lyrics,
    "00s": rap_00_lyrics,
    "10s": rap_10_lyrics,
    "20s": rap_20_lyrics
}

import csv

data_for_csv = []

for decade_name, decade in list_of_lyrics.items():
    i = 0
    for topics in decade:
        i = i + 1
        data_for_csv.append({
            "Method": "BiGrams",
            "Genre": "Pop",
            "Decade": decade_name,
            "Topic": str(decade[topics])
        })
        


# Datei schreiben
path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Analysis_Folder/LDA_BiGrams/"

output_file = path + "topics_rap_LDA_1000_11_1.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
    writer.writeheader()  
    writer.writerows(data_for_csv) 

print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")


In [ ]:
type(rap_70_lyrics[0])

# tipps von ChaGPT
https://chatgpt.com/c/676f1ffc-b134-8012-9f03-6bd466bda148

- Herausfinden, wieviele Topics am geeingesten sind
- warum sind weiterhin so viele Stop words enthalten
- wieviele Lieder nehm ich überhaupt

In [ ]:
print(len(subdataset_80))

In [ ]:
print(len(subdataset_90))

<h1> BERT TOPIC MODELLING </h1>


In [1]:
import gensim
import re
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from gensim.parsing.preprocessing import remove_stopwords
import kagglehub
import os


# Sicherstellen, dass NLTK-Daten heruntergeladen sind
nltk.download('stopwords')
nltk.download('punkt')

# clean the text
import re

def clean_text(document_list):
    
    cleaned_document_list = []
    
    for single_doc in document_list:
        # Delete HTML-Tags
        text = re.sub(r"<.*?>", "", single_doc)
        # Delete Sonderzeichen und Zahlen
        text = re.sub(r"[^a-zA-Z\s]", "", text)
        # Delete uncessary whitespaces
        text = re.sub(r"\s+", " ", text).strip()
        
        # lower case the text
        text = text.lower()
        
        cleaned_document_list.append(text)
    
    return cleaned_document_list

def get_swear_words():

    path = kagglehub.dataset_download("tushifire/ldnoobw")
    path = kagglehub.dataset_download("sahib12/badwords")

    #file = kagglehub.load_dataset("tushifire/ldnoobw",path)

    print("Path to dataset files:", path)

    swear_words_file = os.path.join(path, 'Terms-to-Block.csv')  # Replace 'en.txt' with the correct filename

    with open(swear_words_file, 'r') as f:
        swear_words = set(f.read().splitlines())

    print(f"!!!!Loaded {len(swear_words)} swear words.")

    return swear_words

# Preprocessing: Tokenisieren, Stopwords entfernen, Kleinbuchstaben
def preprocess_stop_words(document_list):
    # Stopwords definieren
    
    stop_words = set(stopwords.words('english'))  # Standard-Stopwords
    additional_words = {"like", "oh", "na", "la", "yo", "you", "get", 
                            "the", "and", "of", "on", "[Intro]", "[Storyteller]"}
    
    swear_words = get_swear_words()
      
    word_set = stop_words.union(additional_words)
    word_set = word_set.union(swear_words)
        
    replacement = ""
    list_of_tokens = []
    for single_doc in document_list:
        # Kombiniere Standard-Stopwords mit zusätzlichen Wörtern

        # Splitte den Text in Wörter
        words = single_doc.split()
        
        # Ersetze jedes Wort, falls es in der Liste vorkommt
        replaced_words = []
        for word in words:
            if word.lower() in word_set:
                replaced_words.append(replacement)  # Ersetze das Wort
            else:
                replaced_words.append(word)  # Behalte das Wort
        
        # Füge den Text wieder zusammen
        list_of_tokens.append(str(" ".join(replaced_words)))
              
    
    return list_of_tokens

def remove_repeated_lines(doc):
    # doc ist eine Liste von Strings, z.B. ["Zeile1", "Zeile2", ...]
    # Wiederholte Zeilen entfernen und Reihenfolge beibehalten
    unique_lines = []
    seen = set()
    for line in doc:
        if line not in seen:
            unique_lines.append(line)
            seen.add(line)
    # Am Ende kannst du wieder eine Liste zurückgeben:
    return unique_lines

def tokenize_all_docs(document_list):
    list_of_tokens = []
    
    for single_doc in document_list:
        
        if type(single_doc) == str:
            tokens = word_tokenize(single_doc)
            list_of_tokens.append(tokens)
        else:
            print("Type von Document", type(single_doc))
            continue
    
    return list_of_tokens

print("Done")


Done


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maurice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maurice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import csv


def processBertTopics(dataset,sentenceTransformer):
    i = 0
    # Initialize your topic model
    #embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    
    embedding_model = SentenceTransformer(sentenceTransformer)
    topic_model = BERTopic(embedding_model=embedding_model)
    # Fit and transform the text data
    #without_repatition = remove_repeated_lines(dataset)
    
    # clean dataset from HTML-Tags, special characters and numbers
    cleaned_song_lyrics = clean_text(dataset)
    
    # Get rid of the bad words and replace it with NOTHING
    swear_words = set(stopwords.words('english'))
    #without_swear = [sort_out_bad_words(doc) for doc in cleaned_song_lyrics]
    
    #Deleting the stopwords
    without_stop_words = preprocess_stop_words(cleaned_song_lyrics)
    
    tokenized_docs = tokenize_all_docs(without_stop_words)
    
    # Convert 2 strings
    tokenized_docs = [" ".join(tokens) for tokens in tokenized_docs]

    # Remove empty strings
    tokenized_docs = [doc for doc in tokenized_docs if doc]
    
    # Remove documents with less than 3 words    
    #tokenized_docs = [doc for doc in tokenized_docs if len(doc.split()) > 3]
    #print("tokenized_docs", tokenized_docs)

    if tokenized_docs == [] or len(tokenized_docs) == 0:
        i = i+1
    else: 
        topics, probs = topic_model.fit_transform(tokenized_docs)

    # Ergebnisse
    #print("Themenzuweisung:", topics)
    #print("Wahrscheinlichkeiten:", probs)

    str_topic_with_probs = []
    #for t, p in zip(topics, probs):
    #    str_topic_with_probs.append(str(t) + ": " + str(p))

    print(i, "documents were empty!")   
    return topic_model.get_topics()

print("Done")


Done


In [16]:
import torch
from transformers import AutoModel, AutoTokenizer

# Modell und Tokenizer laden
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

# Anzahl der Songtexte, die verarbeitet werden sollen
amount = 10  # Beispiel: Anzahl der Texte, die verarbeitet werden sollen

# INPUT TWEET IST BEREITS NORMALISIERT!
# Tokenisierung und Padding/Truncation
lyrics = subdataset_90['lyrics'][:amount]
inputs = tokenizer(lyrics, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Verarbeitung der Eingabe durch das Modell
with torch.no_grad():
    features = bertweet(**inputs)  # Models outputs sind jetzt Tupel

# Ausgabe der Features (optional: erster Layer oder hidden states)
print(features.last_hidden_state)  # Beispielausgabe
print("Done")


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


IndexError: index out of range in self

In [4]:
import time
import datetime
from datasets import load_from_disk
import os
def run_analytics_bert_different_Sentence_Transformers(amount_per_iteration,name_of_run, sentence_transformer):

    start_time = time.time()
    amount = amount_per_iteration

    path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Subdatasets/rap/"

    
    rap_70_lyrics_bert = processBertTopics(load_from_disk(path + 'subdataset_70_rap')['lyrics'][:amount],sentence_transformer)
    rap_80_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_80_rap")['lyrics'][:amount],sentence_transformer)
    rap_90_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_90_rap")['lyrics'][:amount],sentence_transformer)
    rap_00_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_00_rap")['lyrics'][:amount],sentence_transformer)
    rap_10_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_10_rap")['lyrics'][:amount],sentence_transformer)
    rap_20_lyrics_bert = processBertTopics(load_from_disk(path + "subdataset_20_rap")['lyrics'][:amount],sentence_transformer)
    
    
    

    list_of_lyrics_bert = {
        "70s": rap_70_lyrics_bert,
        "80s": rap_80_lyrics_bert,
        "90s": rap_90_lyrics_bert,
        "00s": rap_00_lyrics_bert,
        "10s": rap_10_lyrics_bert,
        "20s": rap_20_lyrics_bert,
    }
    end_time = time.time()

    amount_of_time = end_time - start_time
    
    data_for_csv_bert = []
    
    for decade_name, decade in list_of_lyrics_bert.items():
        i = 0
        
        for topics in decade:
            i = i + 1
            data_for_csv_bert.append({
                "Method": "BERT",
                "Genre": "Rap",
                "Decade": decade_name,
                "Topic": str(decade[topics])
            })
    data_for_csv_bert.append({
                "Method": "TIME",
                "Genre": "TIME",
                "Decade": "TIME",
                "Topic": str(amount_of_time)
            })
    
    # Write to csv
    now = datetime.datetime.now()
    path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Analysis_Folder/BERT/Rap/"
    
    # Datei schreiben
    name_of_file = "topics_RAP_BERT_" + "6"+":"+ "22"+ ":07" + "_MiniLM.csv"
    output_file = os.path.join(path, name_of_file)
    

    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
        writer.writeheader()  
        writer.writerows(data_for_csv_bert) 
   
    print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")
print("Done")


Done


In [5]:
run_analytics_bert_different_Sentence_Transformers(1000,"BERT_mit_stop_words","all-MiniLM-L6-v2")

Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.


/usr/local/lib/python3.10/dist-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


0 documents were empty!
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
0 documents were empty!
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
0 documents were empty!
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
0 documents were empty!
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
0 documents were empty!
Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
0 documents were empty!
CSV-Datei '/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Analysis_Folder/BERT/Rap/topics_RAP_BERT_6:22:07_MiniLM.csv' wurde erfolgreich erstellt!


In [87]:
topic_model.get_topics(0)

NameError: name 'topic_model' is not defined